<a href="https://colab.research.google.com/github/zkaganz/Fake-Real-NEWS/blob/main/Fake_Real_NEWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/deneme/news.csv")

In [ ]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
data.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [ ]:
data.drop(["Unnamed: 0" , "title"],axis=1,inplace=True)

In [ ]:
data["label"] = data["label"].map({"REAL":0,"FAKE":1})

In [ ]:
data.head()

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,U.S. Secretary of State John F. Kerry said Mon...,0
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,It's primary day in New York and front-runners...,0


In [ ]:
sample_rev  = data.text[0] # 0. datamızın ham hali
sample_rev

'Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. \nThe FBI’s leadership is being warned that the entir

In [ ]:
def process(text) :
  text = BeautifulSoup(text).get_text()
  text = re.sub("[^a-zA-Z]"," ", text)
  text = text.lower()
  text = text.split()
  stopw = set(stopwords.words("english"))
  text = [w for w in text if w not in stopw]
  return " ".join(text)

In [ ]:
train_x_tum = []
for r in range(len(data["text"])) :
  if (r+1)%1000 == 0 :
    print("No of text processed = " , r+1)
  train_x_tum.append(process(data["text"][r]))

<ipython-input-9-80baef75c72f>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text).get_text()


No of text processed =  1000
No of text processed =  2000
No of text processed =  3000
No of text processed =  4000
No of text processed =  5000
No of text processed =  6000


In [ ]:
x = train_x_tum
y = np.array(data["label"])
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=44)

In [ ]:

tfidf_vectorizer = TfidfVectorizer(max_features=10000)
x_train = tfidf_vectorizer.fit_transform(x_train).toarray()

In [ ]:
x_train = pd.DataFrame(x_train)

In [ ]:
x_train.shape

(5068, 10000)

In [ ]:
model = LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [ ]:
x_test =tfidf_vectorizer.transform(x_test)
x_test = x_test.toarray()


In [ ]:
x_test.shape

(1267, 10000)

In [ ]:
test_predict = model.predict(x_test)
accur = roc_auc_score(y_test , test_predict)

In [ ]:
print("Doğruluk oranı : " , accur*100)

Doğruluk oranı :  90.9370402692931


In [ ]:

scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)
model.fit(X_train_scaled, y_train)

LogisticRegression()

In [ ]:
test_predict = model.predict(X_test_scaled)
accur = roc_auc_score(y_test , test_predict)

In [ ]:
print("Doğruluk oranı : " , accur*100)

Doğruluk oranı :  93.13489589826706
